In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
img_size = 224
batch_size = 32
epochs = 5
train_path = '/content/drive/My Drive/ASL-recognition/asl_alphabet_train/asl_alphabet_train'

test_path = '/content/drive/My Drive/ASL-recognition/asl_alphabet_test/asl_alphabet_test'

In [ ]:
augment_train_data = ImageDataGenerator(horizontal_flip=True,
                                        rotation_range=50,
                                        zoom_range=0.2,
                                        width_shift_range=0.2,
                                        height_shift_range=0.2,
                                        rescale=1./255,
                                        validation_split=0.1)

augment_test_data = ImageDataGenerator(rescale=1./255)

In [ ]:
train_dataset = augment_train_data.flow_from_directory(train_path,
                                                       shuffle=True,
                                                       classes=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K',
                                                                'L', 'M', 'N',
                                                                'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y',
                                                                'Z', 'space',
                                                                'del', 'nothing'],
                                                       target_size=(
                                                           img_size, img_size),
                                                       batch_size=batch_size)

In [ ]:
test_dataset = augment_train_data.flow_from_directory(test_path,
                                                      classes=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K',
                                                               'L', 'M', 'N',
                                                               'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y',
                                                               'Z', 'space', 'del', 'nothing'],
                                                      target_size=(
                                                          img_size, img_size),
                                                      batch_size=batch_size)

In [ ]:
fig = plt.figure(figsize=(15, 10))
for i in range(1,9):
    plt.subplot(4,2,i)
    plt.imshow(train_dataset[0][0][i-1])
plt.show()

In [ ]:
url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"

download_model = hub.KerasLayer(url,input_shape=(img_size,img_size,3))

model = Sequential([
    download_model,
    Dense(29),
    Activation("softmax")
])

In [ ]:
model.compile(optimizer=Adam(1e-3),
              loss="categorical_crossentropy",
              metrics=['accuracy'])

In [ ]:
print("\n Model summary: ")
print(model.summary())

print("\n Model Training: ")
model.fit(train_dataset,
          batch_size=batch_size,
          epochs=epochs)

In [ ]:
print("\n Model Evaluation: ")
model.evaluate(test_dataset)

In [ ]:
model.save("/content/drive/My Drive/ASL-recognition/h5/asl_model.h5")

In [ ]:
# you can also save the model using this method
# model.save("/content/drive/My Drive/ASL-recognition/model/")

In [ ]:
load_model = tf.keras.models.load_model("/content/drive/My Drive/ASL-recognition/h5/asl_model.h5",custom_objects={"KerasLayer":hub.KerasLayer})
print(load_model.summary())

In [ ]:
# model.predict(test_dataset[0][0])